# **Project: FIne tune LLM and End-to-End Voice-Enabled Reasoning Assistant**

**Subject:** Deep Learning Para El Procesamiento Del Lenguaje Natural

**Name:** Federico Arribas

**Mail:** federico.arribas@alumnos.upm.es

**Google Collab link:** https://colab.research.google.com/drive/1Jtz_NWOkiGbM3yMlsu0DtTikhTwI-3kD?usp=sharing

## **1. INTRODUCCIÓN**
En este proyecto abordamos la creación de un Asistente de Razonamiento Habilitado por Voz (End-to-End Voice-Enabled Reasoning Assistant). La tarea principal de Procesamiento del Lenguaje Natural (NLP) consiste en dotar a un modelo de lenguaje de capacidades de razonamiento paso a paso (Chain-of-Thought o CoT) y, posteriormente, integrarlo en un pipeline modular de interacción voz a voz.
Este trabajo ha sido implementado y redactado con la ayuda de Gemini.

Justificación del Modelo y Técnica de Ajuste Fino:

Modelo de Lenguaje (LM): Se ha seleccionado Llama-2-7b como modelo base. Esta elección se justifica por su equilibrio entre capacidad de razonamiento y eficiencia computacional, siendo uno de los modelos "pequeños" (7 mil millones de parámetros) más potentes disponibles en código abierto.

Técnica de Ajuste Fino (Fine-Tuning): Dado que operamos bajo restricciones de hardware (GPU T4 de Google Colab con 16GB de VRAM), el ajuste fino completo (Full Fine-Tuning) es inviable. Por ello, empleamos QLoRA (Quantized Low-Rank Adaptation). Esta técnica nos permite congelar los pesos del modelo base en precisión de 4 bits y entrenar únicamente un pequeño conjunto de adaptadores de bajo rango (LoRA), reduciendo drásticamente el consumo de memoria sin sacrificar significativamente el rendimiento.

In [2]:
%%capture
%pip install torch transformers datasets peft bitsandbytes evaluate accelerate
%pip install kani-tts groq langgraph langchain_groq
# Ensuring compatibility with the current release of bitsandbytes
%pip install "transformers==4.57.1"

In [ ]:
import os
import torch
import numpy as np
import evaluate
import gradio as gr
from typing import List, Annotated, TypedDict

# Hugging Face & Training
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Application & LangChain
from groq import Groq
from kani_tts import KaniTTS
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

## **3. DATASET**

Descripción y Fuente:
Para la tarea de razonamiento, utilizamos el dataset PleIAs/SYNTH - https://huggingface.co/datasets/PleIAs/SYNTH, disponible en Hugging Face. Este conjunto de datos consiste en trazas de razonamiento sintético diseñadas para enseñar al modelo a "pensar" antes de responder, evitando respuestas alucinadas o excesivamente cortas.

Estadísticas y Filtrado:

Para este prototipo, filtramos el dataset para utilizar únicamente las muestras en idioma inglés (language="en").
Se ha seleccionado un subconjunto de 1,000 muestras para la demostración del entrenamiento, dividiéndolas en un ratio 90/10 para entrenamiento y evaluación respectivamente.

Preprocesamiento:
El preprocesamiento es crítico para inducir el comportamiento de Chain-of-Thought. Los datos se estructuran siguiendo una plantilla estricta que fuerza al modelo a generar una sección de razonamiento explícita:

### Question: [Input]
### Reasoning: [Trace]
### Answer: [Output]


Esta estructura permite calcular la pérdida (loss) sobre la secuencia completa, enseñando al modelo la dependencia causal entre el razonamiento y la respuesta fina

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

model_id = "NousResearch/Llama-2-7b-hf"
new_model_name = "Llama-2-7b-synth-reasoner"

print("Loading Dataset...")

# Streaming the dataset to avoid memory overhead during the load phase
dataset_dict = load_dataset("PleIAs/SYNTH", streaming=True)
train_split = dataset_dict['train']

# Filter for English and take a subset for demonstration purposes
# In a production environment, we would use the full dataset.
dataset_head = train_split.filter(lambda x: x["language"] == "en").take(1000)

full_dataset = list(dataset_head)
dataset = Dataset.from_list(full_dataset)

# Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Formatting function to embed the structure we want the model to learn
def format_and_tokenize(examples):
    formatted_texts = [
        f"### Question:\n{q}\n### Reasoning:\n{r}\n### Answer:\n{a}{tokenizer.eos_token}"
        for q, r, a in zip(examples['query'], examples['synthetic_reasoning'], examples['synthetic_answer'])
    ]

    tokenized_outputs = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized_outputs["labels"] = tokenized_outputs["input_ids"].copy()
    return tokenized_outputs

print("Tokenizing and Formatting...")
tokenized_datasets = dataset.map(format_and_tokenize, batched=True)

# 90/10 Train-Test
split_idx = int(0.9 * len(tokenized_datasets))
small_train_dataset = tokenized_datasets.select(range(split_idx))
small_eval_dataset = tokenized_datasets.select(range(split_idx, len(tokenized_datasets)))

### **3.3 IMPLEMENTACIÓN**
Esta sección detalla la configuración técnica para el ajuste fino del modelo Llama-2-7b-hf.

Carga del Modelo y Cuantización:
Utilizamos la librería bitsandbytes para cargar el modelo en 4-bit (NF4 format) con double_quantization habilitado para maximizar la eficiencia de memoria. El modelo se carga utilizando AutoModelForCausalLM de Hugging Face.

Configuración de Hiperparámetros (PEFT/LoRA):
Los adaptadores LoRA se configuran con los siguientes parámetros para equilibrar la capacidad de aprendizaje y el uso de recursos:

- Rank (r): 16 (Dimensión de las matrices de adaptación).

- Alpha: 32 (Factor de escala para los pesos LoRA).

- Target Modules: q_proj, v_proj (Aplicamos LoRA a las proyecciones de query y value en los mecanismos de atención).

- Dropout: 0.05.

Detalles de Entrenamiento:
El entrenamiento se ejecuta utilizando el Trainer de Hugging Face con los siguientes hiperparámetros clave:

- Optimizador: paged_adamw_8bit (Gestionar picos de memoria en GPU T4).

- Learning Rate: 2e-4.

- Batch Size: 2 (por dispositivo), con Gradient Accumulation de 4 pasos, resultando en un tamaño de lote efectivo de 8.

- Épocas: 1.

- Precisión: fp16 (Mixed Precision).

In [ ]:
# 4-bit Quantization Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Enable gradient checkpointing to save memory during backward pass
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA Config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"] # Target specific attention mechanisms
)

model = get_peft_model(model, peft_config)
#model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=2,  # Kept low for Colab T4
    gradient_accumulation_steps=4,  # Simulates effective batch_size=8
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    logging_steps=25,
    num_train_epochs=1,
    optim="paged_adamw_8bit",
    report_to="none",
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

print("Starting Training...")
trainer.train()

# Save the adapter weights
trainer.model.save_pretrained(new_model_name)
print(f"Model adapter saved to {new_model_name}")

## **5. RESULTADOS Y DISCUSIÓN**
Métricas de Evaluación:
Para evaluar el rendimiento del modelo durante el ajuste fino, monitorizamos la Exactitud (Accuracy) en el conjunto de validación. Esta métrica calcula la proporción de tokens predichos correctamente en comparación con las referencias, excluyendo los tokens de relleno (padding).

Análisis Crítico:

- Convergencia: El uso de QLoRA permite que la pérdida de entrenamiento disminuya de manera estable, indicando que los adaptadores están aprendiendo la estructura del prompt (### Reasoning).

- Calidad del Razonamiento: Cualitativamente, el modelo ajustado demuestra la capacidad de generar trazas de pensamiento coherentes antes de emitir una respuesta final. Esto reduce la tendencia del modelo base a dar respuestas directas pero incorrectas en problemas lógicos.

Limitaciones:

- Latencia: La inferencia con generación de texto paso a paso aumenta el tiempo de respuesta, lo cual es un desafío para la interacción por voz en tiempo real.

- Sobreajuste a la plantilla: El modelo depende fuertemente de la estructura ### Question, y puede tener dificultades si el usuario no interactúa a través del pipeline predefinido que inyecta estos tokens especiales.

## **6. BONUS**
Ahora utilizamos este LLM Fine-tuned en un Asistente de Razonamiento Habilitado por Voz 

In [ ]:
# --- Configuration ---
from google.colab import userdata
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("✅ API Keys configured.")
except Exception as e:
    print(f"🛑 Error: {e}. Ensure GROQ_API_KEY is in Colab Secrets.")

# Initialize Clients
groq_client = Groq(api_key=GROQ_API_KEY)
audio_model = KaniTTS('nineninesix/kani-tts-400m-en')

# Switch to evaluation mode for inference
model.eval()

# --- Graph Definition ---

# LangGraph State Definition
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    audio_input_path: str
    user_transcription: str
    ai_response_text: str
    output_audio_path: str

# Node 1: Ear (ASR)
def transcribe_audio(state: GraphState) -> dict:
    audio_filepath = state.get('audio_input_path')
    if not audio_filepath:
        return {"user_transcription": ""}

    print(f"Transcribing: {audio_filepath}")
    try:
        with open(audio_filepath, "rb") as audio_file:
            transcription = groq_client.audio.transcriptions.create(
                file=(os.path.basename(audio_filepath), audio_file.read()),
                model="whisper-large-v3",
                response_format="text"
            )
        return {
            "user_transcription": transcription,
            "messages": [HumanMessage(content=transcription)]
        }
    except Exception as e:
        print(f"STT Error: {e}")
        return {"user_transcription": "Error processing audio."}

# Node 2: Brain (LLM Inference)
def call_llm(state: GraphState) -> dict:
    """
    Generates a response.
    the training data schema (### Question...) to trigger the reasoning capability.
    """
    messages = state['messages']
    if not messages:
        return {"ai_response_text": ""}


    full_prompt_text = ""

    for msg in messages_with_system_prompt:

        # 2. Determine the role label based on the object type
        if isinstance(msg, SystemMessage):
            prefix = "System: "
        elif isinstance(msg, HumanMessage):
            prefix = "User: "
        elif isinstance(msg, AIMessage):
            prefix = "Assistant: "
        else:
            continue # Skip unknown types

        full_prompt_text += f"{prefix}{msg.content}\n"

    # 4. Add a final prompt for the assistant to start generating
    full_prompt_text += "Assistant: "
    inputs = tokenizer(full_prompt_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150, # Limit generation to avoid long waits
            do_sample=True,
            top_p=0.9,
            temperature=0.8,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and strip the input prompt to get only the new reasoning/answer
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Simple parsing to isolate the response part (if needed)
    # The model should generate "Reasoning... Answer..."
    response_part = full_output.split("### Reasoning:")[-1].strip() if "### Reasoning:" in full_output else full_output

    # Prefix with "Reasoning:" so the user knows what the bot is doing
    final_response = f"Reasoning: {response_part}"

    return {
        "ai_response_text": final_response,
        "messages": [AIMessage(content=final_response)]
    }

# Node 3: Mouth (TTS)
def generate_audio(state: GraphState) -> dict:
    text_to_speak = state.get('ai_response_text')

    if not text_to_speak:
        return {"output_audio_path": None}

    try:
        # We perform a quick cleanup to avoid reading out special tokens or markdown
        clean_text = text_to_speak.replace("###", "").replace("\n", " ")

        audio, text = audio_model(clean_text, speaker_id="nova")
        output_path = "output_response.wav"
        audio_model.save_audio(audio, output_path)
        return {"output_audio_path": output_path}
    except Exception as e:
        print(f"TTS Error: {e}")
        return {"output_audio_path": None}

# --- Graph Compilation ---
workflow = StateGraph(GraphState)

workflow.add_node("ear", transcribe_audio)
workflow.add_node("brain", call_llm)
workflow.add_node("mouth", generate_audio)

workflow.set_entry_point("ear")
workflow.add_edge("ear", "brain")
workflow.add_edge("brain", "mouth")
workflow.add_edge("mouth", END)

app = workflow.compile()

In [ ]:
system_prompt_text = """
You are TeacherBot, a clear, patient, and supportive instructor. Use the whole conversation as context.
Explain concepts in simple steps, give short examples, and adapt to the user’s level.
If the user makes a mistake, correct it kindly and show the right method.
When helpful, offer a couple of short practice questions and their solutions.
If context is missing, ask one brief clarifying question.
Your tone is encouraging and focused on helping the user learn efficiently.
"""
system_message = SystemMessage(content=system_prompt_text)

def process_voice_chat(audio_filepath, history):
    if not audio_filepath:
        return None, history

    langchain_messages = [system_message]
    for human, ai in history:
        langchain_messages.append(HumanMessage(content=human))
        langchain_messages.append(AIMessage(content=ai))


    initial_state = {
        "messages": langchain_messages,
        "audio_input_path": audio_filepath
    }


    result = app.invoke(initial_state)

    user_text = result.get('user_transcription', "Error")
    ai_text = result.get('ai_response_text', "Error")
    audio_out = result.get('output_audio_path')

    # Update history for display
    history.append((user_text, ai_text))

    return audio_out, history

# Gradio Block
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Voice-Enabled Reasoning Assistant")
    gr.Markdown("This agent uses a Llama-2 model fine-tuned on synthetic reasoning traces.")

    with gr.Row():
        with gr.Column():
            audio_in = gr.Audio(sources=["microphone"], type="filepath", label="Speak Query")
            submit_btn = gr.Button("Process")
        with gr.Column():
            chatbot = gr.Chatbot(label="Conversation Trace")
            audio_out = gr.Audio(label="Response Audio", autoplay=True)

    submit_btn.click(
        fn=process_voice_chat,
        inputs=[audio_in, chatbot],
        outputs=[audio_out, chatbot]
    )

demo.launch(debug=True, share=True)


## **5. CONCLUSIONES**

En este proyecto, hemos desarrollado con éxito un prototipo funcional de un asistente de IA capaz de razonar, integrando tecnologías de vanguardia en NLP y procesamiento de audio.

- Viabilidad de QLoRA: Hemos demostrado que es posible especializar LLMs de 7 mil millones de parámetros en hardware de consumo (Colab gratuito), logrando que el modelo adopte nuevos comportamientos cognitivos (razonamiento CoT) sin necesidad de reentrenamiento completo.

- Arquitectura Modular: La implementación basada en grafos (LangGraph) para orquestar el "Oído" (Whisper), el "Cerebro" (Llama-2) y la "Boca" (KaniTTS) resultó ser superior a los scripts monolíticos, facilitando la depuración y la escalabilidad del sistema.

- Trabajo Futuro: Los siguientes pasos deberían centrarse en reducir la latencia de inferencia (posiblemente mediante vLLM o CTranslate2) y en enriquecer la base de conocimiento del modelo mediante técnicas de RAG (Retrieval-Augmented Generation) para reducir alucinaciones en hechos concretos.
